In [29]:
import wntr
import pandas as pd
from datetime import date

In [30]:
inp_file = "C:/Users/kaiorodrigues/Desktop/WNTR/1_Data/Vale_do_Amanhecer_Final_temp.inp"


In [31]:
#Obtem os dados de diametro, tamanho e rugosidade do arquivo:
def Description_pipe(inp_file):
    network = wntr.network.WaterNetworkModel(inp_file)
    pipe_data = {}
    for link_name, link in network.links():
        if isinstance(link, wntr.network.elements.Pipe):
            length = link.length
            diameter = link.diameter
            roughness = link.roughness
            pipe_data[link_name] = {
                "Length (m)": length,
                "Diameter (m)": diameter,
                "Roughness (m)": roughness
            }
    pipe_table = pd.df_descricao_pipes(pipe_data).T
    return pipe_table
#  Realiza o processo de ETL inicial:
def etl_inp(pipe_data):
    pipe_data = pipe_data.reset_index()
    pipe_data.rename(columns={'index': 'ID'}, inplace=True) 
    return pipe_data

#Substituir valores da coluna material (kmeans permite apenas valores numericos)
def mapear_valores(valor):
    if valor == 'PVC':
        return 1
    elif valor == 'PVC DEFOFO':
        return 2
    elif valor == 'PEAD':
        return 3
    elif valor == 'FF':
        return 4
    else:
        return None



In [32]:
pipe_description = Description_pipe(inp_file)
pipe_description_ETL = etl_inp(pipe_description)
pipe_description_ETL

,ID,Length (m),Diameter (m),Roughness (m)
0,0,0.8125,0.25,135.0
1,1,6.2942,0.30,135.0
2,10,226.2101,0.11,132.5
3,100,14.6737,0.06,132.5
4,101,49.1874,0.06,132.5
...,...,...,...,...
581,P21,3.2800,0.10,125.0
582,P31,3.2800,0.30,135.0
583,P33,2.8300,0.30,135.0
584,P35,4.1400,0.20,135.0


In [33]:
Description_material = pd.read_excel("C:/Users/kaiorodrigues/Desktop/WNTR/1_Data/Materiais/Rede_VAM.xlsx")
df_descricao_pipes = Description_material.loc[:, ['ID','DATA_IMPLA','MATERIAL']]
pipe_description_ETL['ID'] = pipe_description_ETL['ID'].str.replace('P', '999')
pipe_description_ETL['ID'] = pipe_description_ETL['ID'].astype(int)

c:\Users\kaiorodrigues\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


------------------

## Unindo os dois dataframes (diamento, comprimento + material idade)

In [39]:
merged_df = pd.merge(pipe_description_ETL, df_descricao_pipes, on='ID', how='left')
merged_df = merged_df.set_index('ID')
merged_df

In [41]:
merged_df.to_excel('C:/Users/kaiorodrigues/Desktop/WNTR/1_Data/Materiais/test.xlsx',index=False)

In [42]:
# substitui os valores NAN por valores especificos
merged_df['DATA_IMPLA'].fillna(value='1996-03-06', inplace=True)
merged_df['MATERIAL'].fillna(value='PVC', inplace=True)

In [43]:
# Converta a coluna 'Data de Nascimento' para datetime
merged_df['DATA_IMPLA'] = pd.to_datetime(merged_df['DATA_IMPLA'])
today = date.today()
merged_df['Idade'] = today.year - merged_df['DATA_IMPLA'].dt.year
merged_df['Idade'] = merged_df['Idade'].fillna(0).astype(int)

In [44]:
merged_df['Material'] = merged_df['MATERIAL'].apply(mapear_valores)

In [45]:
merged_df

,Length (m),Diameter (m),Roughness (m),DATA_IMPLA,MATERIAL,Idade,Material
ID,,,,,,,
0,0.8125,0.25,135.0,1996-03-06,PVC,27,1
1,6.2942,0.30,135.0,1996-03-06,PVC DEFOFO,27,2
10,226.2101,0.11,132.5,1996-03-06,PVC,27,1
100,14.6737,0.06,132.5,1996-03-06,PVC,27,1
101,49.1874,0.06,132.5,1996-03-06,PVC,27,1
...,...,...,...,...,...,...,...
99921,3.2800,0.10,125.0,1996-03-06,PVC,27,1
99931,3.2800,0.30,135.0,1996-03-06,PVC,27,1
99933,2.8300,0.30,135.0,1996-03-06,PVC,27,1


In [47]:
merged_df = merged_df.drop(['DATA_IMPLA', 'MATERIAL'], axis=1)


In [58]:
merged_df = merged_df.drop(['Length (m)'], axis=1)

In [59]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler



# Realizar a padronização dos dados utilizando o StandardScaler
n_clusters = 5

# criar um objeto KMeans e ajustá-lo ao df_descricao_pipes
kmeans = KMeans(n_clusters=n_clusters).fit(merged_df)

# obter as etiquetas de cluster para cada registro
labels = kmeans.labels_

# obter os centróides de cada cluster
centroids = kmeans.cluster_centers_

# imprimir as etiquetas de cluster e os centróides
print("Etiquetas de cluster: ", labels)
print("Centróides: ", centroids)
print(len(labels))

Etiquetas de cluster:  [4 4 0 0 0 0 4 0 4 0 0 0 0 0 0 0 0 0 0 2 0 2 2 2 0 2 2 2 2 2 2 2 2 2 2 0 0 0 0 2 2 0 0 2 2 2 4 2 2 2 2 2 2 0 2 2 2 0 2 0 0 0 0 0 0 2 2 2 0 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 0 2 0 2 0 0 0 0 0 0 0 4 0 2 0 2 2 2 0 0 0 0 0 2 0 4 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 4 4 4 0 0 0 0 0 0 0 0 0 4 4 0 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 4 4 4 2 4 0 0 0 0 0 0 0 4 4 0 2 4 4 4 4 4 4 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 2 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0
 0 0 0 2 0 0 0 0 0 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 4 4 4 4 4 4 4 4 4 4 4 4 0 4 0 4 1 0 0 1 2 1 1 1 1 0 0 0 0 0 2 0 2 2 4 4 0 4 2 2 2 2 0 4 0 0 0 2 2 0 2 2 4 0 0 0 0 4 4 4 0 0 0 0 0 0 0 0 0 0 2 2 0 2 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 2 0 0 0 0 2
 3 3 0 4 0 4 0 4 4 0 0 4 0 4 4 4 0 0 0 0 0 0 4 0 0 0 0 0 0 0 2 4 4 4 4 0 4 4 4 4

In [60]:
labels

array([4, 4, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 4, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 2, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 4, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 2, 4, 4, 4, 4, 4, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 

In [62]:
merged_df['C'] = labels

In [74]:
merged_df['C'].value_counts()

0    323
4    128
2    124
1      7
3      4
Name: C, dtype: int64

In [63]:
merged_df

,Diameter (m),Roughness (m),Idade,Material,C
ID,,,,,
0,0.25,135.0,27,1,4
1,0.30,135.0,27,2,4
10,0.11,132.5,27,1,0
100,0.06,132.5,27,1,0
101,0.06,132.5,27,1,0
...,...,...,...,...,...
99921,0.10,125.0,27,1,0
99931,0.30,135.0,27,1,4
99933,0.30,135.0,27,1,4


----------------------